In [1]:
from vgc.datatypes.Objects import PkmTeam, Pkm, GameState, Weather
from vgc.engine.PkmBattleEnv import PkmBattleEnv
from vgc.behaviour.BattlePolicies import RandomPlayer, TerminalPlayer

In [ ]:
'''
Useful code links
https://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/engine/PkmBattleEnv.py
https://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/datatypes/Constants.py
https://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/behaviour/BattlePolicies.py
'''

### Questions
* How is action space determined
    * DONE 4 moves + num pokemon to switch to
* how do i grab the observation space from the env?
* What happens to the action space when there is a faint and choose to swap to a faint?
* how is reward determined and how should I do it?
    * I am thinking like -1 to 1 with -1 and 1 for winning and losing
        * some points for forcing a faint and damage and inflicting a bad status
        * negative poitns for losing damage, a faint and getting hit with a bad status


### Basic Tutorial example
* render seems to not be working

In [3]:
team0, team1 = PkmTeam(), PkmTeam()
agent0, agent1 = RandomPlayer(), RandomPlayer()
env = PkmBattleEnv((team0, team1),
                   encode=(agent0.requires_encode(), agent1.requires_encode()))  # set new environment with teams
n_battles = 3  # total number of battles
t = False
battle = 0
while battle < n_battles:
    s, _ = env.reset()
    while not t:  # True when all pkms of one of the two PkmTeam faint
        a = [agent0.get_action(s[0]), agent1.get_action(s[1])]
        s, _, t, _, _ = env.step(a)  # for inference, we don't need reward
        #env.render()
    t = False
    battle += 1
print(env.winner)  # winner id number


1


In [6]:
#env.render(mode='ux')
env.render(mode='console')

In [40]:
s

(<vgc.engine.PkmBattleEnv.PkmBattleEnv at 0x105a3b68d90>,
 <vgc.engine.PkmBattleEnv.PkmBattleEnv at 0x105a3aefee0>)

### Exploring env

In [26]:
team0, team1 = PkmTeam(), PkmTeam()
agent0, agent1 = RandomPlayer(), RandomPlayer()
env = PkmBattleEnv((team0, team1),
                   encode=(agent0.requires_encode(), agent1.requires_encode()))  # set new environment with teams
n_battles = 1  # total number of battles
terminated = False
battle = 0
while battle < n_battles:
    obs, _ = env.reset()
    while not terminated:  # True when all pkms of one of the two PkmTeam faint
        action = [agent0.get_action(obs[0]), agent1.get_action(obs[1])]
        obs, reward, terminated, truncated, info = env.step(action)  # for inference, we don't need reward
        #env.render()
    terminated = False
    battle += 1
print(env.winner)  # winner id number


0


In [27]:
reward

[0.875, 1.125]

In [28]:
print(env.observation_space)

Discrete(1188)


#### figuring out action space
* I think it's the 4 current moves plus switching to one of the other pokemon.
    * https://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/behaviour/BattlePolicies.py#L25
    * self.n_actions: int = n_moves + n_switches
* would ahve to test to confirm

In [ ]:
# huh the obs is an env object passed in so getting the action out of it is complicated
# random agent doesn't even use it. just randomizes between n_actions


In [36]:
# so party is two members or three? idk maybe roster is 3 and chose two for the battle?
# team0, team1 = PkmTeam(), PkmTeam()

print(team0.party)
print(team0.fainted())
print(team0.get_pkm_list())
# I guess 3?
print(env.teams[0].size())

[<vgc.datatypes.Objects.Pkm object at 0x00000105A3B2A700>, <vgc.datatypes.Objects.Pkm object at 0x00000105A3B2AD30>]
False
[<vgc.datatypes.Objects.Pkm object at 0x00000105A3B2AB20>, <vgc.datatypes.Objects.Pkm object at 0x00000105A3B2A700>, <vgc.datatypes.Objects.Pkm object at 0x00000105A3B2AD30>]
3


In [16]:
dir(team0)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'active',
 'confused',
 'entry_hazard',
 'fainted',
 'get_not_fainted',
 'get_pkm_list',
 'n_turns_confused',
 'party',
 'reset',
 'reset_team_members',
 'size',
 'stage',
 'switch']

In [12]:
print(env.action_space)

#self.action_space = spaces.Discrete(DEFAULT_N_ACTIONS)

# https://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/datatypes/Constants.py
#DEFAULT_N_ACTIONS = DEFAULT_PKM_N_MOVES + DEFAULT_PARTY_SIZE
# DEFAULT_TEAM_SIZE = 3
# DEFAULT_PARTY_SIZE = DEFAULT_TEAM_SIZE - 1
# DEFAULT_N_MOVES_PKM = 4


Discrete(6)


In [14]:
dir(env.PkmMoveView)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'damage',
 'recover',
 'set_entry_hazard',
 'set_fixed_damage',
 'set_recover',
 'set_stage',
 'set_status',
 'set_weather']

In [10]:
dir(env)

['PkmMoveView',
 '_PkmBattleEnv__create_pkm_move_view',
 '_PkmBattleEnv__get_attack_dmg_rcvr',
 '_PkmBattleEnv__get_attack_order',
 '_PkmBattleEnv__get_entry_hazard_damage',
 '_PkmBattleEnv__get_fixed_damage',
 '_PkmBattleEnv__get_forward_env',
 '_PkmBattleEnv__get_post_battle_damage',
 '_PkmBattleEnv__get_pre_combat_damage',
 '_PkmBattleEnv__get_recover',
 '_PkmBattleEnv__get_states',
 '_PkmBattleEnv__perform_pkm_attack',
 '_PkmBattleEnv__process_post_battle_effects',
 '_PkmBattleEnv__process_pre_battle_effects',
 '_PkmBattleEnv__process_switch_pkms',
 '_PkmBattleEnv__switch_fainted_pkm',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__se

In [32]:
dir(env.teams[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'active',
 'confused',
 'entry_hazard',
 'fainted',
 'get_not_fainted',
 'get_pkm_list',
 'n_turns_confused',
 'party',
 'reset',
 'reset_team_members',
 'size',
 'stage',
 'switch']

In [34]:
env.teams[0].size()

3

In [35]:
env.teams[1].size()

3